Testing notebook for RNN for tinder profile generation

Based on Char RNN trump project

In [0]:
import sys
import numpy as np
import string

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation

#Change Layer count parameter as needed

LAYER_COUNT = 4
HIDDEN_LAYERS_DIM = 512


# generic vocabulary
characters = list(string.printable)
characters.remove('\x0b')
characters.remove('\x0c')

VOCABULARY_SIZE = len(characters)
characters_to_ix = {c:i for i,c in enumerate(characters)}
print("vocabulary len = %d" % VOCABULARY_SIZE)
print(characters)

Using TensorFlow backend.


vocabulary len = 98
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ', '\t', '\n', '\r']


In [0]:
#Load different models according to each layer, pick one for each approach

#1 Layer
#!gdown https://drive.google.com/uc?id=102P1bDW6Zw2HG0RySW-lerQNk4D8kUZJ
#base_dir ='basic_LSTM_tindergenv1_singlelayer).h5'
#2 Layers
#!gdown https://drive.google.com/uc?id=1W5MaWWLVlevtAEYbO3ZdeeFsj5Z1PGL3
#base_dir ='basic_LSTM_tindergenv1_2layers.h5'
#4 Layers
#!gdown https://drive.google.com/uc?id=1Ntbzl9j5YB_fNWyKlH9PyMJAEhqRuHyW
#base_dir ='basic_LSTM_tindergenv1_4layers.h5'

#4layers 20chars
!gdown https://drive.google.com/uc?id=1gn2agW0W7arYZNOfQWHQ8wxxSbFItgma
base_dir = 'basic_LSTM_tindergenv1_4layers_20char.h5'


Downloading...
From: https://drive.google.com/uc?id=1gn2agW0W7arYZNOfQWHQ8wxxSbFItgma
To: /content/basic_LSTM_tindergenv1_4layers_20char.h5
91.2MB [00:00, 197MB/s]


In [0]:
#Load in model weights, we changed our input count to 1 for generation (vs. 10 for training)

test_model = Sequential()
for i in range(LAYER_COUNT):
    test_model.add(
            LSTM(
                HIDDEN_LAYERS_DIM, 
                return_sequences=True if (i!=(LAYER_COUNT-1)) else False,
                batch_input_shape=(1, 1, VOCABULARY_SIZE),
                stateful=True
            )
        )
test_model.add(Dense(VOCABULARY_SIZE))
test_model.add(Activation('softmax'))
test_model.compile(loss='categorical_crossentropy', optimizer="adam")

from keras.models import load_model
test_model.load_weights(base_dir)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Auxilliary methods from Chollet

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array"""
    # from fchollet/keras
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def predict_next_char(model, current_char, diversity=1.0):
    """Predict the next character from the current one"""
    x = np.zeros((1, 1, VOCABULARY_SIZE))
    x[:,:,characters_to_ix[current_char]] = 1
    y = model.predict(x, batch_size=1)
    next_char_ix = sample(y[0,:], temperature=diversity)
    next_char = characters[next_char_ix]
    return next_char

def generate_text(model, seed, count=250):
    """Generate characters from a given seed"""
    model.reset_states()
    for s in seed[:-1]:
        next_char = predict_next_char(model, s)
    current_char = seed[-1]

    sys.stdout.write("["+seed+"]")
    
    for i in range(count - len(seed)):
        next_char = predict_next_char(model, current_char, diversity=0.5)
        current_char = next_char
        sys.stdout.write(next_char)
    print("...\n")

In [0]:
for i in range(50):
    generate_text(
        test_model,
        seed="I like"
    )

[I like] makeup and showing my artistic side. Im funny and goofy  I love cooking Im not into the hook up culture as much as people my age. Im sick of a single mother to a conversation going to the gym and my dog type  I like to do a lot but what I was ...

[I like] to go on new places, living in a boating and watching greys an activities and trying new things.  If you swipe right on insta again.SextonI love animals and marvel movies  I'm very sarcastic with a dirty sense of humor  I am not skinny I am a ...

[I like] dancing and girly drinks and trying new things.  If you have a dog named Tinkerbelle that i wont have a driver license.I like dancing and girly drinks and trying new things.  If you swipe right, good to know someone besides my mom thinks Im a ...

[I like] to travel and go on adventures! Im a mom. I love going to concerts and trying new things.  If you swipe right, good to get to know me and that all color change I am the best conversation and see where things go on and wa